# EDA 5: Pokročilejší manipulace a agregace


Další čtení pro dlouhé večery v tomto ročním období:
* https://github.com/alanderex/pydata-pandas-workshop/tree/master/notebooks
* https://github.com/TomAugspurger/pandas-head-to-tail/blob/master/notebooks/03-Iterators-Groupby.ipynb
* https://realpython.com/pandas-groupby/

In [ ]:
import pandas as pd
%matplotlib inline

## Analýza dat po skupinách

Jedním ze základních postupů v datové analýze je rozdělení dat do skupin, aplikace nějaké operace na jednotlivé skupiny a nakonec kombinace výsledků do vhodného datasetu. Anglicky se tento postup označuje jako *split-apply-combine*. Skupiny jsou často, byť ne nutně, definovány nějakou (kategorickou) proměnnou, např. by to mohla být barva, pohlaví nebo kontinent. Skupiny lze ale vytvářet i odvozováním, např. pomocí rozsahu nebo nebo vlastností časových řad. Oboje už jsme vlastně viděli v EDA 3, kdy jsme skupiny vytvářeli pro deštivé dny nebo jednotlivé roky.

Pojďme si to vysvětlit prakticky. Použijeme k tomu hezký dataset s výsledky voleb do českého horní komory parlamentu, do Senátu ČR. Data pochází z [webu ČSÚ](https://volby.cz/opendata/senat_vse/senat_vse_opendata.htm). Na tomto webu najdeme i [podrobnou dokumentaci](https://volby.cz/opendata/senat_vse/SENATregPopis.pdf).

In [ ]:
import os
filename = "serk.csv"
if not os.path.exists(filename):
    filename = "https://raw.githubusercontent.com/PyDataCZ/pyladies-kurz/main/lessons/pydata/pandas_groupby/serk.csv"
df = pd.read_csv(filename, delimiter=";", encoding="windows-1250", parse_dates=["DATUMVOLEB"])

In [ ]:
df.head(3).T

### Krok 1: Split

Na rozdělení dat do skupin slouží metoda [`groupby`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html?highlight=groupby#pandas.DataFrame.groupby). Nejjednodušší a možná i nejčastější použití je seskupení podle existujícího sloupce v tabulce. Třeba podle `OBVOD` nebo `DATUMVOLEB`.

In [ ]:
districts = df.groupby("OBVOD")
districts

Dostali jsme objekt typu [`GroupBy`](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html). Pokud vám to připomíná `DatetimeIndexResampler` z [Explorativní analýzy a statistiky jedné proměnné](https://naucse.python.cz/2020/pydata-praha-jaro/pydata/eda-univariate-timeseries/), je to velice dobrá asociace. Tato operace totiž data jen rozdělila do skupin, ještě jsme ale neřekli, co s těmi skupinami chceme dělat.

### Krok 2 + 3: Apply + Combine

Na skupiny, které jsme vytvořili pomocí `groupby`, teď můžeme aplikovat nějakou operaci. To je právě krok *apply*. Nemá pro nás moc cenu oddělovat tento krok od třetího kroku *combine*, protože když už nějakou opraci aplikujeme, tak bychom rádi výsledek sestavili do výsledného datasetu. Pandas navíc tyto kroky sám nijak neodděluje.

*Pozn.: Apply a combine začne být více odděleno v nástrojích na zpracování velkých dat, které už se nevejdou pohodlně do operační paměti počítače a pandas na ně už nestačí. Apply pak probíhá po částech, třeba i distribuovaně na oddělených serverech, a výsledné combine se provádí sesbíráním částečných výsledků.* 

Použijeme teď jednoduchou agregační metodu `count`, která nám vrátí počet hodnot (po skupinách samozřejmě).

In [ ]:
districts.count()

Na levé straně v indexu vidíme skupiny, podle kterých se dataset agregoval a do sloupců se daly všechny sloupce, na které bylo možné aplikovat naši agregační funkci (v tom případě na všechny zbylé).

Z čísel si můžeš všimnout, že u některých chybí údaje o titulech.

Než aplikujeme krok *apply*, můžeme si vybrat, na který ze sloupců tak učiníme, trochu si tím zpřehledníme výstup. Pokud si vybereme jen jeden sloupec, dostaneme Series.

In [ ]:
districts["JMENO"].count()

Tento krok si můžeme trochu zjednodušit - než abychom se doptávali na `count` nad jedním sloupcem v rámci agregace, můžeme se doptat na *velikost* každé ze skupin.

In [ ]:
districts.size()

In [ ]:
df.head(3)

**Úkol:** Rozděl data podle data voleb (`DATUMVOLEB`) a vypiš počet záznamů v každé skupině. Dokážeš výsledek setřídit podle velikosti skupin? 

Možná sis všimla, jaký je u výsledku index. Pokud ne, nevadí, určitě si všimneš teď. Zkusíme totiž vytvořit skupiny ne z jednoho sloupce, ale ze dvou. Pojďme si rozdělit volby podle data a obvodu.

In [ ]:
date_district_counts = df.groupby(["DATUMVOLEB", "OBVOD"]).count()
date_district_counts

Máme tedy skupiny, které jsou definované dvojicí hodnot datum a obvod (`DATUMVOLEB`, `OBVOD`). A to je přesně důvod, proč existuje v Pandas [`MultiIndex`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html).

## Co je ten `DataFrameGroupBy object`?

Na začátku jsme si udělali základní agregaci bez aplikace funkcí a dostali jsme jakýsi objekt. Je možné s ním něco dělat, aniž bychom agregovali? Ukazuje se, že ano.

Než se k tomu dostaneme, zkusme jeden **úkol**: Rozděl náš dataframe podle stran - tzn. pro každou stranu vytvoř dataframe a ten ulož do zvláštního souboru. Např. `Democrat` půjde do Democrat.csv atd.

In [ ]:
df.groupby('DATUMVOLEB')

Jedna klíčová funkce, kterou nám tento objekt nabízí, je iterace.

In [ ]:
groups = df.groupby('DATUMVOLEB')

next(iter(groups)) # timhle ziskame prvni element pri iteraci (for cyklu)

K čemu nám to může být? Při agregaci se data drasticky zjednodušují a nemusíme si vždy být jisti, že naše agregace jsou napsané správně. Pomocí iterace nad skupinami si můžeme zobrazit všechna data před agregací.

In [ ]:
for date, sdf in df.groupby('DATUMVOLEB'):
    sdf.to_csv(str(date) + '.csv')

Tuto funkcionalitu asi tolik neoceníte, když jde jen o jeden sloupec, zde `DATUMVOLEB`, tedy alternativa není tak složitá. Ale jakmile začnete agregovat nad více sloupci, začne být iterace čím dál užitečnější.

## Další agregační funkce

Dosud jsme agregovali jen pomocí `.size` nebo `.count`, ale existuje spousta další agregačních metod, zejm. těch numerických.

Pro snadnější práci s agregacemi budeme používat metodu `.agg`, která akceptuje slovník. Tento slovník udává, *co* chceme agregovat (klíč) a *jak* to chceme agregovat (hodnota). S tím, že způsobů agregace pro jeden sloupec může být více naráz. Ukážeme si.

In [ ]:
df.groupby('DATUMVOLEB').agg({'VEK': 'max'}).head()

In [ ]:
df.groupby('DATUMVOLEB').agg({
    'VEK': ['min', 'max'],
    'VSTRANA': 'nunique'}
).head()

## Transformace sloupců

Většinu času jsme pracovali s daty, které jsme načetli ze souboru a krom nějakého základního čištění jsme je žádným způsobem neměnili. Teď si ukážeme, jak udělat některé základní transformace.

Může nás například zajímat, které hodnoty máme či nemáme v daném sloupci. K získání takové informace sloužít metody `.isnull` a `.notnull`, které jsou navzájem inverzní.

In [ ]:
df['POVOLANI'].isnull()

Series má metodu `isnull`, která nám vrátí True/False hodnoty podle toho, jestli daná hodnota chybí nebo ne (NULL v SQL). Pro snadnější pochopení je možné použít inverzní metodu `notnull`.

Může náš též zajímat, zda řádky nabývají některou z vybraných hodnot.

In [ ]:
df['JMENO'].isin(['Jan', 'Jana'])

... nebo jak se hodnoty liší mezi řádky (dává smysl jen pro číselná data nebo pro sloupce obsahující datum)

In [ ]:
df['VEK'].diff()

### .str a .dt metody

Můžeme též testovat různé vlastnosti stringů a dat (od slova datum). Slouží k tomu [.str](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#string-methods) a [.dt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html).

In [ ]:
df['PRIJMENI'].str.len()

Často používanými metodami v rámci `.str` je `contains` nebo `.lower/upper`

In [ ]:
names = pd.Series(['JOHN', 'Jean-Luc', 'Mary-Jane', 'Kate', 'John'])
names.str.contains('-')

In [ ]:
names[names.str.contains('-')]

Zatímco `.str.contains` použijeme zpravidla na filtrování, `.lower` poslouží třeba na unifikaci dat, která se pak lépe agregují (a deduplikují).

In [ ]:
names.value_counts()

In [ ]:
names.str.lower().value_counts()

Z `.dt` si ukážeme jak vytáhnout z data rok.

In [ ]:
election_years = df['DATUMVOLEB'].dt.year

election_years#.value_counts()

**Úkol:** vyfiltruj kandidáty, kterým v době kandidatury bylo 40 let

## Agregace nad nesloupci

Dosud jsme vždy agregovali nad něčím, co jsme měli v DataFramu. Je ale možné agregovat nad daty, které tam vůbec nemáme, alespoň ne explicitně. K tomu budou sloužit transformace, které jsme si právě ukázali.

Kromě názvu sloupce můžeme do `groupby` vložit nějakou Series (!), která má stejný tvar jako naše sloupce a pandas podle toho bude umět agregovat. Jaká taková Series se nabízí? Nejlépe transformace nějakého existujícího sloupce.

Můžeme tak agregovat data na základě _měsíce_, kdy se dané volby konaly. A to aniž bychom tento sloupec přidávali do dataframu. Tato metoda nám tak pomůže dělat kreativní agregace bez nutnosti měnit naše data.

In [ ]:
df.groupby([
    df['DATUMVOLEB'].dt.month,
]).size()

Můžeme samozřejmě přidávat další a další (ne)sloupce.

In [ ]:
df.groupby([
    df['OBVOD'],
    df['DATUMVOLEB'].dt.month,
]).size()

Měsíce jsou fajn, je jich jen 12, ale co když budeme chtít agregovat nad roky voleb?

In [ ]:
df.groupby([
    df['OBVOD'],
    df['DATUMVOLEB'].dt.year,
]).size().count()

Dostáváme poněkud velký dataset, který nám toho moc neřekne. S tím nám pomůže další kapitola.

### Binning

Sice máme věky lidí, ale přeci jen jich je spousta a moc nám to neřekne, určitě se podle nich nedá dobře agregovat.

In [ ]:
df['VEK'].hist(bins=25)

In [ ]:
df['VEK'].value_counts().head()

Ani histogram, ani `value_counts` nám žádné moc hodnotné informace nepřinesl. Budeme muset data trochu seskupit. Na to v pandas existuje několik možností.

První z nich je nám již známý `.value_counts`, kterému můžeme přihodit argument `bins`, který znamená, že nechceme frekvence jednotlivých hodnot, ale že chceme seskupit data do několika intervalů.

Podívejme 

In [ ]:
df['VEK'].value_counts(bins=10)

Co když nám takové samorozdělení nestačí? Na to je `pandas.cut`, resp. `pd.cut`. Má spoustu možností, doporučujeme [projít dokumentaci](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html).

In [ ]:
bins = [20, 30, 40, 50, 60, 70, 80, 90, 100]
pd.cut(df['VEK'], bins)

In [ ]:
pd.cut(df['VEK'], bins).value_counts()

Nevýhodou `pd.cut` je, že intervaly jsou určeny uživatelem a může chvíli trvat, než je člověk odladí. Více automatická je možnost určit intervaly pomocí statistického rozložení dat, k tomu slouží `pd.qcut`, ten místo hranic intervalů bere kvantily.

In [ ]:
pd.qcut(df['VEK'], [0, .1, .5, .9, 1]).value_counts()

_Samozřejmostí tohoto seskupování je, že tyto transformované sloupce opět můžeme použít pro agregaci._

**Úkol**: Jak se postupem času měnil počet kandidujících lidí důchodového věku (65 let a více)? Spočítej počet lidí pod 65, nad 65 a nad 90 v každém roce, kdy se konaly volby do senátu.

### Kontingenční tabulky

Pokud budeme agregovat nad několika sloupci, může se nám stát, že z DataFramu se nám stane jedna dlouhá nudle. V tom se nedá moc dobře vyznat. Pokud jste s takovými daty někdy pracovali v Excelu, možná vám bude povědomá funkce kontigenčních tabulek, v angličtině _pivot tables_.

Než začneme pivotovat, vytvořme si malý dataset - bude to jednoduchá agregace na základě roku voleb a obvodu, ve kterém se kandidovalo.

In [ ]:
summary = df.assign(ROK=df["DATUMVOLEB"].dt.year)
summary = summary.groupby(['ROK', 'OBVOD'], as_index=False).size()
summary = summary.rename(columns={'size': 'count'})
summary

Součástí každého `pivot`u jsou tři faktory - specifikace,

1. který se sloupců chceme v indexu
2. který bude ve sloupcích
3. který sloupec se přiřadí do hodnot v DataFramu

Hezky je to vidět [na tomto diagramu](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html).

In [ ]:
summary.pivot(index='OBVOD', columns='ROK', values='count').fillna("")

Omezením metody `pivot` je to, že umí jen otáčet DataFrame, ale pokud některému z políček odpovídá více hodnot, vyhodí vám chybu, protože neví, jak je má agregovat. Pivot je opravdu jen pro otáčení.

Na komplexnější agregace tu je podobně pojmenovaný `pivot_table`.

In [ ]:
summary = df.assign(ROK=df["DATUMVOLEB"].dt.year)
summary = summary.groupby(['OBVOD', 'ROK', 'TITULPRED'], as_index=False).size()
summary = summary.rename(columns={'size': 'count'})
summary

`pivot_table` nabízí možnost přiřadit do některé z dimenzí (do indexu či sloupců) vícero sloupců, a následně pak vytvoří víceúrovňový index. Jde též specifikovat, jaká funkce se aplikuje, pokud na buňku připadá více hodnot (jako v Excelu).

Více detailů najdete [v dokumentaci](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html).

In [ ]:
wide = summary.pivot_table(index=['OBVOD', 'TITULPRED'], columns=['ROK'], values='count', fill_value='')

wide.head(10)

Kdyby ti přišlo matoucí, že tu jsou dvě podobné funkce na kontigenční tabulky, tak tě ještě víc zmateme, protože existuje ještě `pd.crosstab`. Do detailů zde už zacházet nebudeme, doporučujeme [dokumentaci](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html).

### Inverzní kontigenční tabulky

Zatímco na kontigenční tabulky mnozí narazili, jejich inverzní funkcionalita je celkem neznámá, a zajímavá. Funguje přesně tak, jak píšeme - vezme se široká tabulka, ve které je jedna z dimenzí ve sloupcích, a zúží a prodlouží se tím, že se ony názvy sloupcí překonvertují do samotného sloupce. Příklad bude názornější.

V pandas se na tuto inverzní operaci používá metoda `melt`. Její delší výčet argumentů je tradičně [v dokumentaci](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html).

Mějme dataset, kde je výkon v různých regionech fiktivní země. Nevýhodou je, že nemůžeme úplně dívat na časové řady, protože roky, ke kterým údaje platí, jsou ve sloupcích, ne v řádcích. Takto široké tabulky jsou celkem populární třeba u dat o počasí.

In [ ]:
df = pd.DataFrame({
    'region': ['North', 'South', 'East', 'West'],
    '2000': [200, 100, 50, 1000],
    '2005': [450, 10, 510, 1040],
    '2010': [10, 500, 950, 500],
    '2015': [550, 20, 50, 10],
    '2020': [1, 1, 5, 10],
})
df

`pd.melt` zajímá, který ze sloupců nějak identifikuje dané řádky (`id_vars`) a které sloupce obsahují hodnoty (`value_vars`). Zbylé dva argumenty slouží jen k přejmenování sloupců (`var_name`, `value_name`).

In [ ]:
long = df.melt(id_vars=['region'], value_vars=['2000', '2005', '2010', '2015', '2020'],
        var_name='year', value_name='output')

long

**Úkol:** vem tento dlouhý dataset a udělej z něj ten původní, široký. (Neřeš, pokud máš někde index, kde před tím nebyl.)